# Forecasting com a biblioteca Prophet
Artigo sobre o Prophet: https://peerj.com/preprints/3190/<br>
Documentação: https://facebook.github.io/prophet/docs/quick_start.html#python-api

In [11]:
import MySQLdb
import decouple
import pandas as pd
import gspread
import gspread_dataframe as gd
import time

from tqdm.auto import tqdm
from decouple import Config, RepositoryEnv
from datetime import datetime
from fbprophet import Prophet #as novas versoes utlizam apenas from prophet import Prophet

In [12]:
## Configurando credenciais: dw e google sheets
config = decouple.Config(RepositoryEnv('/home/marlon/Documents/prophet/.env'))
gc = gspread.service_account(filename = 'key.json')
sh = gc.open_by_key(config('CODE_SHEET'))

## Extraindo os dados do Data Warehouse (DW)

In [13]:
def query_dw(query):
    con = MySQLdb.connect(host=config('DW_HOST'),
                         user=config('DW_USER'),
                         passwd=config('DW_PASSWORD'),
                         db='dw')
    dw = con.cursor()

    dw.execute(query)

    colunas = []
    for i in dw.description:
        colunas.append(i[0])

    return pd.DataFrame(list(dw.fetchall()), columns=colunas)

In [14]:
def gerar_dados_completos():
    print('Generating data...')
    inicio = time.time()
    dados = query_dw(
    """
        select 
            date,
            business_model,
            sum(gmv) as gmv,
            sum(tickets) as ticket
        from ft_results
        where year(date) >= 2017
        group by 1, 2
        order by 1
    """
    )
    dados['ano'] = pd.DatetimeIndex(dados['date']).year
    dados['mes'] = pd.DatetimeIndex(dados['date']).month
    
    gd.set_with_dataframe(worksheet=sh.worksheet('Dados'), dataframe=dados, include_index=False, \
                          include_column_header=True, resize=True)
    
    total = dados.groupby('date', as_index=False)[['gmv', 'ticket']].sum()
    total.columns = ['date', 'gmv_total', 'ticket_total']

    pivot = pd.pivot_table(data=dados, index='date', columns=['business_model'], values=['gmv', 'ticket'])
    pivot = pivot.reset_index()
    pivot = pivot.dropna(axis=1)
#     print(pivot)
    pivot.columns = ['date', 'gmv_ota', 'gmv_outras_otas', 'gmv_parc', 'gmv_wl', 'ticket_ota', 'ticket_outras_otas','ticket_parc', 'ticket_wl']
    pivot = pivot.drop(columns=['gmv_outras_otas','ticket_outras_otas'])
    
    join = pd.merge(pivot, total, on='date')
    join.to_csv('data/gmv_full.csv', sep=',', index=False)
    fim = time.time()
    
    print(f'Data generated! Time: {fim - inicio} s')

## Aplicando Prophet

In [15]:
def high_season(ds):
    date = pd.to_datetime(ds)
    return date.month == 12 or date.month == 7 or date.month == 2

In [16]:
def modelo(dados, metrica, data_corte, holidays):
    dados['ds'] = pd.to_datetime(dados['ds'])
    
    # Removendo dos dados o período da queda de vendas na pandemia
    dados.loc[(dados['ds'] > '2020-03-14') & (dados['ds'] < '2020-07-01'), 'y'] = None

    # Aplicando o regressor da high season
    dados['high_season'] = dados['ds'].apply(high_season)
    
    # Modelo preditivo 
    modelo = Prophet(holidays                = holidays,
                     daily_seasonality       = False, 
                     weekly_seasonality      = True, 
                     yearly_seasonality      = True,
                     interval_width          = 0.8,
                     n_changepoints          = 25,
                     changepoint_range       = 0.85,
                     changepoint_prior_scale = 0.05,
                     holidays_prior_scale    = 10.0,
                     seasonality_prior_scale = 5.0, 
                     seasonality_mode        = 'multiplicative')
    modelo.add_regressor('high_season')
    
    modelo.fit(dados[dados['ds'] <= data_corte])
    
    return modelo

In [17]:
def previsao(modelo, data_corte, data_previsao):
    # Definindo o período para previsão
    data_corte = datetime.strptime(data_corte, "%Y-%m-%d")
    data_previsao = datetime.strptime(data_previsao, "%Y-%m-%d")
    period = data_previsao - data_corte
    
    data_futuro = modelo.make_future_dataframe(periods=period.days)
    data_futuro['high_season'] = data_futuro['ds'].apply(high_season)
    previsao = modelo.predict(data_futuro)
    
#     previsao['yhat'] = previsao['yhat'] * 1.05 #apply(lambda x: x*1.05) 
    
    return previsao

In [18]:
# Variaveis a serem modificadas #
data_inicio_previsao = '2023-10-01'
data_fim_previsao = '2023-11-30'
gerar_dados = 1 # caso o dataset esteja desatualizado ou não exista, 1 para gerar ou atualizar e 0 caso contrário

In [19]:
metricas = ['gmv_total', 'gmv_ota', 'gmv_parc', 'gmv_wl', 'ticket_total', 'ticket_ota', 'ticket_parc', 'ticket_wl']

calendar = pd.read_table('data/calendar.tsv')
holidays = calendar[(calendar['ds'] >= '2017-01-01') & (calendar['ds'] <= '2023-12-31')]

if gerar_dados:
    gerar_dados_completos()

dataset = pd.read_csv('data/gmv_full.csv')
dt_cal = dataset['date'].iloc[-1]    # data até onde tem dados reais; ou pode ser definida uma outra data especifica

for metrica in tqdm(metricas):
    dados = dataset[['date', metrica]]
    dados.columns = ['ds', 'y']
    
    model = modelo(dados, metrica, dt_cal, holidays) 
    forecast = previsao(model, dt_cal, data_fim_previsao)
    
    forecast = forecast[['ds', 'yhat_lower', 'yhat_upper', 'yhat']]
    forecast = forecast[forecast['ds'] >= data_inicio_previsao]
    
    dados['ds'] = pd.to_datetime(dados['ds']) # necessario para fazer o merge
    dados = dados[['ds', 'y']]
    
    forecast = pd.merge(forecast, dados, how='left', on='ds')
    
    gd.set_with_dataframe(worksheet=sh.worksheet(metrica), dataframe=forecast, include_index=False, \
                          include_column_header=True, resize=True)
    
#     forecast.to_csv('previsoes/'+metrica+'.csv', sep=',', index=False)

Generating data...


Data generated! Time: 131.45418286323547


  0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_19571/4163482035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['ds'] = pd.to_datetime(dados['ds'])
/tmp/ipykernel_19571/4163482035.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['high_season'] = dados['ds'].apply(high_season)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

Initial log joint probability = -31.577
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        5920.8     0.0386707       1128.95           1           1      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       6034.27    0.00525219       490.305           1           1      244   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       6110.81     0.0699698       645.978      0.2338           1      367   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       6162.71    0.00156779       490.884      0.7683      0.7683      490   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       6176.98   0.000172263       245.066      0.8118      0.8118      613   
    Iter      log prob        ||dx||      ||grad||       alpha   

/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.appen

...

/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.appen

Initial log joint probability = -20.8904
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       5879.73     0.0182917       1081.16        1.26       0.126      121   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       5954.06     0.0218244       1305.24      0.4836           1      247   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        5976.1   0.000855823       219.527           1           1      361   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       5985.14    0.00222518       101.821           1           1      488   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       5986.95    0.00326942       110.899      0.8807     0.08807      611   
    Iter      log prob        ||dx||      ||grad||       alpha  

/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.appen

Initial log joint probability = -56.1225
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        5494.1    0.00840164       669.671           1           1      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       5595.82     0.0457801       1055.88           1           1      230   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        5637.6     0.0010597       642.409      0.2423      0.2423      345   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       5656.58     0.0160684       342.665           1           1      464   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       5674.68   0.000517353       110.707      0.9195      0.9195      590   
    Iter      log prob        ||dx||      ||grad||       alpha  

/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/home/marlon/.local/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.appen